## MCP4728 Digitial to Analog Converter  
The MCP4728 is a digitial to analog converter.  

Manufacturer Link:  
https://www.microchip.com/en-us/product/MCP4728  

Breakout Board Used:  
https://learn.adafruit.com/adafruit-mcp4728-i2c-quad-dac  

### Notes on use

In most I2C cases, v_dd will be either 3.3V or 5.0V. The MCP4728 can
handle as much as 24mA current at 5V (0.12W) in short circuit. 
By comparison, the Raspberry Pi can source at most 16mA of current 
at 3.3V (0.05W). Unless the application output will draw a very 
small amount of current, an external (to the I2C bus) voltage source 
should probably be used. 
See the Adafruit ISO1540 Bidirectional I2C Isolator as a possible solution.

The manufacturer uses the term VDD (Vdd) for external voltage, many other 
sources use the term VCC (Vcc). VDD is used here to be consistent with the
datasheet, but manufacturers like Adafruit use VCC on the pinouts labels.

In [1]:
from meerkat import mcp4728, parser

In [2]:
m = mcp4728.MCP4728(bus_n=1)

In [3]:
r = m.general_call_read_address()

Setting the VDD / VCC voltage supplied to the chip is required

In [4]:
m.v_dd = 3.3

### 1. Determining input code from voltage  
For a first case, let's create a 0.5V output.  

In [5]:
m.calculate_input_code?

Signature: m.calculate_input_code(v_target, v_ref_source, gain, v_dd)
Docstring:
Calculate the required input register code
required to produce a specific voltage

Parameters
----------
v_target : float, voltage target output on the DAC
v_ref_source : str, either 'internal' (2.048V/4.096V) or 'external' (VDD)
gain : int, gain of DAC, either 1 or 2
v_dd : float, voltage source for the device. Could be I2C 3.3V or 5.0V, 
    or something else supplied on VDD

Returns
-------
int, DAC inpute code required to achieve v_target
File:      ~/code/meerkat/meerkat/mcp4728.py
Type:      function


In [6]:
m.calculate_input_code(v_target=0.5, v_ref_source='internal', gain=1, v_dd=3.3)

1000

It's not possible to generate a voltage above the reference voltage.

In [7]:
m.calculate_input_code(v_target=2.35, v_ref_source='internal', gain=1, v_dd=3.3)

'Gain must be 2 for v_target > v_ref internal'

In [8]:
m.calculate_input_code(v_target=2.35, v_ref_source='internal', gain=2, v_dd=3.3)

2350

Confirm the code with generate the desired output voltage. Note `v_dd` is the VDD/VCC voltage supplied to the chip, either from the I2C bus or via an isolated external voltage source.

In [9]:
m.output_voltage?

Signature: m.output_voltage(input_code, v_ref_source, gain, v_dd=None)
Docstring:
Check output code voltage output

Parameters
----------
input_code : int, DAC inpute code required to achieve v_target
v_ref_source : str, either 'internal' (2.048V/4.096V) or 'external' (VDD)
gain : int, gain of DAC, either 1 or 2
v_dd : float, voltage source for the device. Could be I2C 3.3V or 5.0V, 
    or something else supplied on VDD
Returns
-------
v_target : float, DAC vol
File:      ~/code/meerkat/meerkat/mcp4728.py
Type:      function


In [10]:
m.output_voltage(input_code=2350, v_ref_source='internal', gain=2, v_dd=3.3)

2.35

Using the internal reference, the target voltage cannot be above 2x 2.048V = 4.096V, even if the chip is supplied with 5.0V power.

In [11]:
m.calculate_input_code(v_target=5.0, v_ref_source='internal', gain=2, v_dd=5.0)

'v_target must be <= 4.096V if using internal'

Finally, if using an external voltage source the target voltage still has to be at or below the supplied voltage.

In [12]:
m.calculate_input_code(v_target=5.5, v_ref_source='external', gain=1, v_dd=5.0)

'v_target must be <= v_dd'

In [13]:
m.calculate_input_code(v_target=5.5, v_ref_source='external', gain=2, v_dd=5.0)

'v_target must be <= v_dd'

### 2. Channel A Demo  
Let's get 0.5V from channel A

In [14]:
m.calculate_input_code(v_target=0.5, v_ref_source='internal', gain=1, v_dd=3.3)

1000

And check the code will do what we want

In [15]:
m.output_voltage(input_code=1000, v_ref_source='internal', gain=1, v_dd=3.3)

0.5

Now program the channel using `set_channel`

In [16]:
m.set_channel?

Signature:
m.set_channel(
    channel,
    v_ref_source,
    power_down,
    gain,
    input_code,
    description='no description',
)
Docstring:
Write single channel output

Parameters
----------
channel : str, either 'a', 'b', 'c' or 'd' corresponding 
    to the output channel
v_ref_source : str, either 'internal' (2.048V/4.096V) or 'external' (VDD)
power_down : str, either 'normal' or one of the power-down
    resistor to ground values of '1k' '100k' or '500k'
gain : int, either 1 or 2 for multiplier relative to
    the internal reference voltage
input_code : int, between 0 and 4095 to set the 
    output voltage
File:      ~/code/meerkat/meerkat/mcp4728.py
Type:      method


In [17]:
m.set_channel(channel='a',
              v_ref_source='internal',
              power_down='normal',
              gain=1,
              input_code=1000,
              description='test 1: 0.5V')

Using a multi-meter, confirm the voltage between the pins `VA` (+) and `GND` (-) is 0.5

### 3. Channel B Demo  
This time we'll use a gain of 2, even though we don't need to. The LSB resolution will be lower in this case but note the `v_target` is above the 2.048 of v_ref when gain is 1. By setting the gain to 2, v_ref is now 4.096.

In [18]:
m.calculate_input_code(v_target=3.1, v_ref_source='internal', gain=2, v_dd=3.3)

3100

In [19]:
m.output_voltage(input_code=3100, v_ref_source='internal', gain=2, v_dd=3.3)

3.1

In [20]:
m.set_channel(channel='b',
              v_ref_source='internal',
              power_down='normal',
              gain=2,
              input_code=3100,
              description='test 2: 3.1V')

### 4. Data Output Formats  
The class attribute `state` stores each channels commanded state since code initialization. It does not read the registers directly. Therefore if the chip has been powered on but the Python class reinitialized, this state will be out of sync until each channel is issued a command again.

In [21]:
m.state

{'a': ['test 1: 0.5V', 'a', 'internal', 3.3, 'normal', 1, 1000, 0.5],
 'b': ['test 2: 3.1V', 'b', 'internal', 3.3, 'normal', 2, 3100, 3.1],
 'c': None,
 'd': None}

Publish the state to JSON

In [22]:
m.publish()

['{"description": "test 1: 0.5V", "channel": "a", "v_ref_source": "internal", "v_dd": 3.3, "power_down": "normal", "gain": 1, "input_code": 1000, "output_voltage": 0.5, "std_time_ms": "2021-12-23 15:38:10.805738"}',
 '{"description": "test 2: 3.1V", "channel": "b", "v_ref_source": "internal", "v_dd": 3.3, "power_down": "normal", "gain": 2, "input_code": 3100, "output_voltage": 3.1, "std_time_ms": "2021-12-23 15:38:10.805884"}',
 '{"description": "not_initialized", "channel": "c", "v_ref_source": null, "v_dd": null, "power_down": null, "gain": null, "input_code": null, "std_time_ms": "2021-12-23 15:38:10.806010"}',
 '{"description": "not_initialized", "channel": "d", "v_ref_source": null, "v_dd": null, "power_down": null, "gain": null, "input_code": null, "std_time_ms": "2021-12-23 15:38:10.806083"}']

Write the data to disk, in this case using the default writer format of csv

In [23]:
m.write()

In [24]:
parser.csv_resource(m.csv_writer.path)

({'encoding': 'utf-8',
  'format': 'text/csv',
  'standard': 'Follow RFC 4180',
  'line_terminator': '\n',
  'quote_char': '"',
  'double_quote': True,
  'escape_char': '\\',
  'null_sequence': 'NA',
  'comment': '#',
  'path': '2021_12_23_15_38_10_mcp4728.csv',
  'time_source': 'std_time_ms',
  'time_format': '%Y-%m-%d %H:%M:%S.%f',
  'delimiter': ',',
  'skip_initial_space': True,
  'case_sensitive_header': False,
  'skip_lines': 1},
                   std_time_ms      description channel v_ref_source  v_dd  \
 0  2021-12-23 15:38:10.826521     test 1: 0.5V       a     internal   3.3   
 1  2021-12-23 15:38:10.826785     test 2: 3.1V       b     internal   3.3   
 2  2021-12-23 15:38:10.826973  not_initialized       c         None  None   
 3  2021-12-23 15:38:10.827133  not_initialized       d         None  None   
 
   power_down  gain input_code  output_voltage              datetime64_ns  
 0     normal     1       1000             0.5 2021-12-23 15:38:10.826521  
 1     normal   